# Samples cleaning

This task will fetch the raw data from the remote database given a SQL query file.

## Import libraries

In [1]:
import sys

sys.path.append('..')

In [2]:
import os
import gc
import yaml
import json
import requests
import numpy as np
import pandas as pd

from pymongo import MongoClient

from utils import get_files, memory_usage, \
correct_encoding, update_page_metadata

## Load the data

In [3]:
BASE_PATH = '../../'

CONFIG_DIR = os.path.join(BASE_PATH, 'config')
STORAGE_DIR = os.path.join(BASE_PATH, 'storage')
DATA_DIR = os.path.join(BASE_PATH, 'data')

config = yaml.load(open(os.path.join(CONFIG_DIR, 'env.yml')),
                   Loader=yaml.FullLoader)

In [4]:
model = 'samples'
model_storage = os.path.join(STORAGE_DIR, model)

if not os.path.exists(model_storage):
    os.makedirs(model_storage)
    
client = MongoClient('mongodb://{}:{}@{}:{}'.format(config['MONGO_USERNAME'], config['MONGO_PASSWORD'],
                                                    config['MONGO_HOST'], config['MONGO_PORT']))
metadata_db = client[config['MONGO_DATABASE']]

model_metadata = metadata_db[model]

model_metadata.delete_many({})
model_metadata.insert_one({'pages': []})

In [5]:
csv_files = get_files(os.path.join(DATA_DIR, 'raw', model, '*.csv'))

In [6]:
if not csv_files:
    raise FileNotFoundError('Couldn\'t find any csv files! Please make sure the filepath exists')

In [7]:
df = pd.read_csv(csv_files[0], sep=';', parse_dates=['timestamp'], nrows=100)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 38 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id                    100 non-null    int64         
 1   device_id             100 non-null    int64         
 2   timestamp             100 non-null    datetime64[ns]
 3   battery_state         100 non-null    object        
 4   battery_level         100 non-null    float64       
 5   timezone              100 non-null    object        
 6   country_code          100 non-null    object        
 7   memory_active         100 non-null    int64         
 8   memory_inactive       100 non-null    int64         
 9   memory_free           100 non-null    int64         
 10  memory_user           100 non-null    int64         
 11  charger               100 non-null    object        
 12  health                100 non-null    object        
 13  voltage              

In [8]:
df.describe(include=[np.number])

,id,device_id,battery_level,memory_active,memory_inactive,memory_free,memory_user,voltage,temperature,usage,...,roaming_enabled,bluetooth_enabled,location_enabled,power_saver_enabled,nfc_enabled,developer_mode,free,total,free_system,total_system
count,100.000000,100.000000,100.000000,1.000000e+02,100.000000,1.000000e+02,100.000000,100.000000,100.00000,100.000000,...,100.0,100.0,100.000000,100.0,100.000000,100.0,100.000000,100.000000,100.000000,100.000000
mean,50.500000,1.870000,71.380000,6.753474e+05,483018.360000,1.574720e+06,65265.840000,4.119500,28.77100,0.320900,...,0.0,0.0,0.710000,0.0,0.290000,0.0,4098.300000,10567.510000,460.100000,2699.510000
std,29.011492,1.368144,31.909208,2.883390e+05,46542.844283,2.086183e+05,11685.824609,0.181578,4.09235,2.246707,...,0.0,0.0,0.456048,0.0,0.456048,0.0,332.706001,1468.018581,278.189293,1085.850339
min,1.000000,1.000000,5.000000,4.161320e+05,401032.000000,1.442060e+06,46156.000000,3.790000,21.40000,-21.430000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,3789.000000,9634.000000,27.000000,1009.000000
25%,25.750000,1.000000,40.750000,4.913160e+05,417181.000000,1.442060e+06,58796.000000,3.930000,27.30000,0.030000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,3919.000000,9634.000000,27.000000,1009.000000
50%,50.500000,1.000000,88.000000,5.039200e+05,504364.000000,1.442060e+06,62174.000000,4.175000,27.70000,0.695000,...,0.0,0.0,1.000000,0.0,0.000000,0.0,3921.000000,9634.000000,637.000000,3390.000000
75%,75.250000,4.000000,94.000000,1.111631e+06,515056.000000,1.899508e+06,67296.000000,4.290000,32.02500,0.875000,...,0.0,0.0,1.000000,0.0,1.000000,0.0,4611.000000,12853.000000,637.000000,3390.000000
max,100.000000,4.000000,100.000000,1.141176e+06,545980.000000,1.899508e+06,117900.000000,4.320000,37.20000,2.210000,...,0.0,0.0,1.000000,0.0,1.000000,0.0,4611.000000,12853.000000,637.000000,3390.000000


In [9]:
df.describe(exclude=[np.number])

,timestamp,battery_state,timezone,country_code,charger,health,network_status,network_type,mobile_network_type,mobile_data_status,mobile_data_activity,wifi_status
count,100,100,100,100,100,100,100,100,100,100,100,100
unique,98,3,2,2,2,1,3,3,1,2,3,1
top,2017-10-08 12:50:04,Charging,America/Chicago,us,ac,Good,lte,MOBILE,lte,disconnected,none,enabled
freq,2,74,71,71,76,100,36,36,100,64,57,100
first,2017-10-08 12:50:04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,2017-10-09 03:51:20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df.sample(10, random_state=0)

,id,device_id,timestamp,battery_state,battery_level,timezone,country_code,memory_active,memory_inactive,memory_free,...,roaming_enabled,bluetooth_enabled,location_enabled,power_saver_enabled,nfc_enabled,developer_mode,free,total,free_system,total_system
26,27,1,2017-10-09 03:32:07,Charging,92.0,America/Chicago,us,515912,492896,1442060,...,0,0,1,0,0,0,3789,9634,637,3390
86,87,4,2017-10-08 20:41:32,Charging,23.0,Atlantic/Madeira,pt,1138884,415404,1899508,...,0,0,0,0,1,0,4611,12853,27,1009
2,3,1,2017-10-08 12:55:15,Discharging,81.0,America/Chicago,us,500276,501200,1442060,...,0,0,1,0,0,0,3922,9634,637,3390
55,56,1,2017-10-08 16:40:32,Charging,92.0,America/Chicago,us,485352,498540,1442060,...,0,0,1,0,0,0,3917,9634,637,3390
75,76,4,2017-10-08 20:21:42,Charging,9.0,Atlantic/Madeira,pt,1112576,415600,1899508,...,0,0,0,0,1,0,4611,12853,27,1009
93,94,4,2017-10-08 20:59:59,Charging,36.0,Atlantic/Madeira,pt,1132436,407380,1899508,...,0,0,0,0,1,0,4611,12853,27,1009
16,17,1,2017-10-08 13:37:00,Charging,92.0,America/Chicago,us,506832,501396,1442060,...,0,0,1,0,0,0,3921,9634,637,3390
73,74,4,2017-10-08 20:19:57,Charging,7.0,Atlantic/Madeira,pt,1110596,415372,1899508,...,0,0,0,0,1,0,4611,12853,27,1009
54,55,1,2017-10-08 16:36:24,Charging,91.0,America/Chicago,us,474960,510160,1442060,...,0,0,1,0,0,0,3918,9634,637,3390
95,96,4,2017-10-08 21:06:29,Charging,40.0,Atlantic/Madeira,pt,1134200,408072,1899508,...,0,0,0,0,1,0,4611,12853,27,1009


## Basic data pre-processing

In [11]:
def save_df_page(page, chunks, collection, exclude=['id']):
    # concatenate data chunks -> careful benchmark this
    df = pd.concat(chunks, axis=0, ignore_index=True)
    
    collection.update_one({}, {'$inc': {'nrows': df.shape[0]}})

    # store page metadata in mongo
    update_page_metadata(collection, df)

    # save output to a parquet file with brotli compression
    df.to_parquet(os.path.join(model_storage, '{}.{}.parquet'.format(
        model, page)), compression='brotli', index=False)
    
    del df

    return page + 1

In [12]:
# projected memory usage for all samples loaded
round(memory_usage(df) * 5000000 / df.shape[0], 2)

4500.0

In [13]:
mappings = {'id': 'uint32', 'device_id': 'uint32', 'battery_level': 'uint8', 'memory_active': 'uint32',
            'memory_inactive': 'uint32', 'memory_free': 'uint32', 'memory_user': 'uint32', 'usage': 'uint8',
            'screen_on': 'bool', 'wifi_signal_strength': 'int16', 'wifi_link_speed': 'int16',
            'screen_brightness': 'int16', 'roaming_enabled': 'bool', 'bluetooth_enabled': 'bool',
            'location_enabled': 'bool', 'power_saver_enabled': 'bool', 'nfc_enabled': 'bool',
            'developer_mode': 'bool', 'free': 'uint32', 'total': 'uint32', 'free_system': 'uint32',
            'total_system': 'uint32', 'wifi_enabled': 'bool', 'mobile_enabled': 'bool', 'wifi_active': 'bool',
            'mobile_active': 'bool'}

model_metadata.update_one({}, {'$set': {'mappings': mappings}})

In [14]:
settings_columns = ['screen_on', 'roaming_enabled', 'bluetooth_enabled', 'location_enabled',
                    'power_saver_enabled', 'nfc_enabled', 'developer_mode', 'wifi_enabled',
                    'mobile_enabled', 'wifi_active', 'mobile_active']

# left to right binary representation
int16_repr = lambda x: x.dot(1 << np.arange(x.shape[-1])).astype(np.uint16)

In [15]:
total_rows = 0
total_memory_usage = 0.0
memory_usage_split = 0.0
last_id = 0

page = 0
chunks = []

features = df.columns.to_list()
features = list(filter(lambda x: x not in ['country_code'], features))

string_columns = df[features].select_dtypes(include='object').columns.to_list()

exclude_columns = ['network_status', 'network_type', 'mobile_network_type',
                  'mobile_data_status', 'mobile_data_activity', 'wifi_status']

category_columns = list(set(string_columns) - set(exclude_columns))

In [16]:
print('Total csv files: {}'.format(len(csv_files)))

Total csv files: 165


In [17]:
for filepath in csv_files:
    df = pd.read_csv(filepath, sep=';', usecols=features,
                     parse_dates=['timestamp'])

    # drop missing values
    df = df.dropna()

    # process features which can drop values first
    df = df[df['timestamp'].between(pd.Timestamp(
        2017, 9, 1), pd.Timestamp(2020, 8, 1))]

    # battery level
    df = df[df['battery_level'] <= 100]

    # temperature
    lower, upper = df['temperature'].quantile([0.01, 0.99])
    df = df[df['temperature'].between(lower, upper)]

    # voltage
    df.loc[df['voltage'] > 1000, 'voltage'] = df['voltage'] / 1000

    # cpu usage
    df.loc[:, 'usage'] = abs(df['usage']) * 100
    df.loc[df['usage'] > 100, 'usage'] = df['usage'] / 100
    df = df.drop(df[df['usage'] > 100].index, axis=0)

    # basic string pre-processing: convert to lowercase and strip blank chars
    for column in string_columns:
        df.loc[:, column] = [x.lower().strip() for x in df[column].tolist()]

    df['wifi_enabled'] = 0
    df.loc[df['wifi_status'].isin(
        ['enabled', 'enabling']), 'wifi_enabled'] = 1

    df['mobile_enabled'] = 0
    df.loc[df['network_type'].str.startswith(
        'mobile'), 'mobile_enabled'] = 1

    df['wifi_active'] = 0
    df.loc[(df['network_status'] == 'wifi') | (
        df['network_type'].str.startswith('wifi')), 'wifi_active'] = 1

    df['mobile_active'] = 0
    df.loc[df['mobile_enabled'] & (df['network_status'] != 'disconnected') & (
        df['mobile_data_status'].isin(['connecting', 'connected'])), 'mobile_active'] = 1

    df = df.drop(exclude_columns, axis=1)

    df['profile'] = int16_repr(df[settings_columns].values)
    
    df[category_columns] = df[category_columns].astype('category')
    df = df.astype(mappings)

    total_rows += df.shape[0]
    page_memory_usage = memory_usage(df)
    total_memory_usage += page_memory_usage
    memory_usage_split += page_memory_usage

    last_id = max(last_id, df['id'].max())

    chunks.append(df)

    if memory_usage_split >= config['MEMORY_USAGE_SPLIT']:
        print('Page {} created!'.format(page))
        page = save_df_page(page, chunks, model_metadata)

        del chunks[:]
        del chunks
        gc.collect()

        chunks = []
        memory_usage_split = 0.0

    print('{} ✔'.format(filepath))

../../data/raw/samples/samples.query.1.csv ✔
../../data/raw/samples/samples.query.2.csv ✔
../../data/raw/samples/samples.query.3.csv ✔
Page 0 created!
../../data/raw/samples/samples.query.4.csv ✔
../../data/raw/samples/samples.query.5.csv ✔
../../data/raw/samples/samples.query.6.csv ✔
../../data/raw/samples/samples.query.7.csv ✔
Page 1 created!
../../data/raw/samples/samples.query.8.csv ✔
../../data/raw/samples/samples.query.9.csv ✔
../../data/raw/samples/samples.query.10.csv ✔
../../data/raw/samples/samples.query.11.csv ✔
Page 2 created!
../../data/raw/samples/samples.query.12.csv ✔
../../data/raw/samples/samples.query.13.csv ✔
../../data/raw/samples/samples.query.14.csv ✔
../../data/raw/samples/samples.query.15.csv ✔
Page 3 created!
../../data/raw/samples/samples.query.16.csv ✔
../../data/raw/samples/samples.query.17.csv ✔
../../data/raw/samples/samples.query.18.csv ✔
../../data/raw/samples/samples.query.19.csv ✔
Page 4 created!
../../data/raw/samples/samples.query.20.csv ✔
../../dat

Page 39 created!
../../data/raw/samples/samples.query.164.csv ✔
../../data/raw/samples/samples.query.165.csv ✔


In [18]:
if chunks:
    save_df_page(page, chunks, model_metadata)
    del chunks[:]
    del chunks
    gc.collect()

In [19]:
model_metadata.update_one({}, {'$set': {'last_id': int(last_id)}})
model_metadata.update_one({}, {'$set': {'total': total_rows}})

In [20]:
total_rows, total_memory_usage, last_id

(46558859, 5046.179999999999, 49490940)

In [21]:
content = '**{}** | task: data pre-processing, rows: {}, memory usage: {} MB' \
.format(model, total_rows, total_memory_usage)

requests.post(config['discord_webhook_url'], data={'content': content})

<Response [204]>